# Методы предобучения без учителя, ФКН ВШЭ

## Домашнее задание 2: контрастные методы предобучения

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после указанного срока сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

Это задание посвящено контрастным self-supervised методам. Мы остановимся на алгоритмах SimCLR и BYOL, которые разбирались на занятиях. Эксперименты будем проводить на наборе данных [STL10](https://cs.stanford.edu/~acoates/stl10/), который идеально подходит для предобучения без разметки, поскольку содержит 100к неразмеченных, 5к обучаюших и 8к тестовых изображений.

Для сдачи задания необходимо провести эксперименты, описанные в этом ноутбуке, и написать о них отчет в формате PDF. Вместе с отчетом сдается код, позволяющий запустить эксперименты. Прежде чем что-то имплементировать, прочитайте все постановки экспериментов и подумайте, как лучше организовать код, не забудьте также зачекпойнтить необходимые обученные модели. Мы оставляем за собой право снижать оценку за плохо структурированный код. **Обязательно** используйте оптимизации для обучения из [списка](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html) в своих пайплайнах, например, Automatic Mixed Precision, чтобы ускорить запуски. Также **обратите внимание**, что отчет &mdash; это обязательная составляющая оценки, без него мы не будем проверять ваше задание. Отчет обязательно должен включать кривые обучения для всех моделей, которые вы запускаете. Кроме того, следите за тем, чтобы ваши рисунки и графики были читаемыми, и по возможности красивыми :)

## Задание 0. Supervised baseline

**0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов**

Начнем наше исследование с обучения supervised модели из случайного начального приближения. Используйте размеченный train для обучения, и размеченный test для тестирования. В качестве архитектуры нейронной сети будем использовать ResNet-18\*. Советуем поискать гиперпараметры и аугментации для обучения на STL-10 по статьям. У автора задания получилось accuracy в районе 71-72%.

\**Для наборов данных с меньшем размерешением, чем у ImageNet (таких как CIFAR-10/100, STL-10), принято использовать ResNet-18 с измененными первыми слоями: обычно свертка 7x7 заменяется на 3x3, а 2x2 MaxPooling выбрасывается. Мы предлагаем вам учить обычный ResNet-18 в реализации torchvision, чтобы сэкономить время.* 

## Задание 1. SimCLR

**1.5 балла**

Реализуйте и обучите метод SimCLR из статьи [Chen et al, 2020](https://arxiv.org/pdf/2002.05709.pdf). Мы хотим, что вы реализовали его самостоятельно, поэтому в этом задании запрещается заимствовать код из открытых источников. В качестве обучающей выборки возьмите неразмеченную часть STL-10, для валидации алгоритма используйте размеченный train.

## Задание 2. BYOL

**2.5 балла**

Аналогично предыдущему заданию, реализуйте и обучите метод BYOL из статьи [Grill et al, 2020](https://arxiv.org/pdf/2006.07733.pdf). Для того, чтобы проверить, что проекции не коллапсируют (переменная $z$ из исходной статьи), вместе с лоссом отрисовывайте стандартное отклонение $z$. 

## Задание 3. t-SNE

**1.5 балла**

С помощью метода t-SNE визуализируйте эмбеддинги изображений из обучающей и тестовой выборок, которые получаются у supervised, SimCLR и BYOL моделей. В качестве эмбеддингов принимается выход average pooling'а модели. Точки, соответствующие каждому из 10 классов, должны быть покрашены в свой цвет. 

## Задание 4. Linear probing.

**1 балл**

Обучите линейную пробу для self-supervised моделей, сравните качество с supervised обучением из случайной инициализации.

## Задание 5. Fine-tuning.

**1.5 балла**

Дообучите self-supervised модели на классификацию STL-10. Если вы все сделали правильно, то качество должно получиться на несколько процентов выше, чем у бейзлайна. Аналогично заданию 3, нарисуйте, как изменялись эмбеддинги изображений по ходу процесса файн-тюнинга. 

## Задание 6. OOD robustness.

**2 балла**

Здесь и далее мы будем сравнивать между собой 5 моделей:
- Supervised
- SimCLR + linear probing
- SimCLR + fine-tuning
- BYOL + linear probing
- BYOL + fine-tuning

Будем сравнивать модели по устойчивости на out-of-distribution объектах. В качестве OOD набора данных возьмем тестовую выборку CIFAR-10, которая имеет практически такие же классы, как STL-10 (9/10 классов). При этом, если модель предсказала класс "monkey", которого нет в CIFAR-10, то будем считать, что ее предсказание &mdash; класс "frog". Сравните обученные модели по OOD accuracy. 

## Бонус 1. Optimum width.

**2 балла**

В научных кругах существует поверье, что способность модели к генерализации (то есть к высокому качеству на тестовой выборке) связана с геометрией лосса в окрестности модели. Если вкратце, то считается, что широкие оптимумы (а наша финальная модель &mdash; это оптимум лосса по обучающей выборке) генерализуются лучше, чем узкие. В этом эксперименте мы предлагаем вам разобраться с метриками ширины оптимумов, выбрать ту, что приходится вам по душе, и оценить ее на 5 обученных моделях. В отчете вам нужно объяснить, как вычисляется выбранная вами метрика и чем она мотивирована. Прилагаем вам статьи для вдохновения: [Keskar et al, 2017](https://arxiv.org/pdf/1609.04836.pdf), [Jiang et al, 2019](https://arxiv.org/pdf/1912.02178.pdf), [Dinh et al, 2017](https://arxiv.org/pdf/1703.04933.pdf).

## Бонус 2. MoCo

**2 балла**

В качестве бонуса рассмотрим еще одну контрастную self-supervised модель, MoCo v2, из статей [He et al, 2019](https://arxiv.org/pdf/1911.05722), [He et al, 2020](https://arxiv.org/pdf/2003.04297.pdf). Проведите все описанные выше эксперименты с ней.